<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Moder Times con dos niveles de coste
* Calculamos pasosA, pasosB y total para el primer nivel y también para el segundo nivel.
* El segundo nivel se alcanza haciendo el supuesto de que se hubiera tomado la decisión de bajar (de A a B) uno de los números, que proporciona un cierto coste y luego, en el siguiente número a bajar también se calcula su coste.
* Finalmente se baja el que menor coste tenga sumando el total de los dos niveles.
* El cálculo del segundo nivel no es necesario hacerlo, salvo que en el primer nivel el de coste más bajo tenga un valor mayor o igual a 10
 - en ese caso, se analizarían todos los de mayor coste y el siguiente
 - por ejemplo, los costes del nivel 1 son: 14 20 14 15 15 18 32
 - podríamos analizar un segundo nivel de los dos de 14 y los dos de 15

  
* Únicamente se inicia el algoritmo pasando los dos primeros elementos de la pila A a la pila B: ```a[0],a[1]```

In [48]:
# FUNCIONES
def sa(a, b, result):   # result es un array que va acumulando los pasos dados. Ejemplo: result = ['pb','ra','sa','rra']
    if len(a) > 1: a[0],a[1] = a[1],a[0]; result.append('sa')
    return a, b, result
def sb(a, b, result):
    if len(b) > 1: b[0],b[1] = b[1],b[0]; result.append('sb')
    return a, b, result
def ss(a, b, result):   # ss no llama a sa y sb ya que en result quedarían anotadas las tres funciones: ss, sa y sb
    if len(a) > 1 or len(b) > 1:
        result.append('ss')
        if len(a) > 1: a[0],a[1] = a[1],a[0]
        if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a, b, result
def pa(a, b, result):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0); result.append('pa')
    return a, b, result
def pb(a, b, result):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0); result.append('pb')
    return a, b, result
def ra(a, b, result):
    if len(a) > 1: a.append(a.pop(0)); result.append('ra')
    return a, b, result
def rb(a, b, result):
    if len(b) > 1: b.append(b.pop(0)); result.append('rb')
    return a, b, result
def rr(a, b, result):
    if len(a) > 1 or len(b) > 1:
        result.append('rr')
    if len(a) > 1: a.append(a.pop(0))
    if len(b) > 1: b.append(b.pop(0))
    return a, b, result
def rra(a, b, result):
    if len(a) > 1: a.insert(0, a.pop()); result.append('rra')
    return a, b, result
def rrb(a, b, result):
    if len(b) > 1: b.insert(0, b.pop()); result.append('rrb')
    return a, b, result
def rrr(a, b, result):
    if len(a) > 1 or len(b) > 1:
        result.append('rrr')
    if len(a) > 1: a.insert(0, a.pop())
    if len(b) > 1: b.insert(0, b.pop())
    return a, b, result

In [ ]:
# PASOS NECESARIOS PARA COLOCAR CADA ELEMENTO DE A EN SU SITIO EN B      total = pasosA + pasosB (esta suma es solo una idea, se ha de sumar de una forma peculiar)

def necesariosA(a, b):   # array pasosA calcula los pasos necesarios para colocar cada elemento de A como el primero de su pila
    for v in a:
        if a.index(v) < len(a)/2:
            pasosA.append(a.index(v))
        else:
            pasosA.append(-(len(a)- a.index(v)))

def necesariosB(a, b):   # array pasosB calcula los pasos de B necesarios para colocar cada elemento de A dentro de su sitio en B
    for i in range(len(a)):    # objetivo_primero es el número que se ha de poner en la primera posición de la pila B
        if a[i] < min(b):      # si el elemento de A considerado es menor que el mayor de B entonces
            objetivo_primero = max(b)   # el objetivo_primero será el mayor de B
        else:   # objetivo_primero en este caso será el maximo de los inferiores en B al valor iésimo de A
            objetivo_primero = min(b)   #se inicializa en el valor mínimo de la pila B
            for j in range(len(b)):
                if b[j] < a[i] and b[j] > objetivo_primero:
                        objetivo_primero = b[j]
        # el objetivo_primero se ha de situar el primero de la pila B            
        if b.index(objetivo_primero) < len(b)/2:
            pasosB.append(b.index(objetivo_primero))
        else:
            pasosB.append(-(len(b)- b.index(objetivo_primero)))

def totaliza(a, b):   # totalizar pasos
    global total
    for i in range(len(pasosA)):
        if pasosA[i] * pasosB[i] < 0:   # si son de distinto signo, uno positivo y otro negativo
            total.append(abs(pasosA[i]) + abs(pasosB[i]))   # no hay sinergia
        else:   # si son de igual signo o alguno cero
            total.append(max(abs(pasosA[i]), abs(pasosB[i])))  # si son de igual signo hay sinergia
'''
def calculaIndexPasosMinimos():
    global pasosA, pasosB, total
    pasosA = []   # [0,1,2, ...,41,-41,... , -2,-1]  vector donde cada index está asociado con el valor del mismo index en A
    pasosB = []   # [-3,2,-5, ..., -5,0]   estos dos arrays se han de recalcular cada vez que realmente se mueva algún elemento de A a B
    total = []
    necesariosA(a, b)
    necesariosB(a, b)
    totaliza(a, b)
    return total.index(min(total))   # retorna el índice del elemento de la pila A que menos pasos totales necesita
'''

In [50]:
def calculaIndexPasosMinimos():
    # Nivel 1
    global pasosA, pasosB, total
    pasosA = []   # [0,1,2, ...,41,-41,... , -2,-1]  vector donde cada index está asociado con el valor del mismo index en A
    pasosB = []   # [-3,2,-5, ..., -5,0]   estos dos arrays se han de recalcular cada vez que realmente se mueva algún elemento de A a B
    total = []
    necesariosA(a, b)
    necesariosB(a, b)
    totaliza(a, b)
    all_min = [i for i, x in enumerate(total) if x == min(total)]   # da un array con todos los index del valor mínimo
    #min1 = all_min[-1]   #min(total)   # mínimo número de pasos totales del Nivel 1
    #if min1 >= 0:
        #print("pasosA", pasosA)
        #print("pasosB", pasosB)
        #print(min1, total)
    return all_min[-1]   #total.index(min1)   # retorna el índice del elemento de la pila A que menos pasos totales necesita

In [51]:
def giraA(a, b, indice, steps, result):
    for i in range(abs(steps)):
        if steps > 0:
            ra(a, b, result)
        elif steps < 0:
            rra(a, b, result)

def giraB(a, b, indice, steps, result):
    for i in range(abs(steps)):
        if steps > 0:
            rb(a, b, result)
        elif steps < 0:
            rrb(a, b, result)

# girando pilas A y B
def giraPilas(a, b, indice, result):   # indice es el index del valor en la pila A que deseamos poner el primero
    if pasosA[indice] * pasosB[indice] > 0:   # Existe sinergia, nos podemos ahorrar pasos
        pasos_comunes = min(abs(pasosA[indice]), abs(pasosB[indice]))
        for i in range(pasos_comunes):
            if pasosA[indice] > 0:   # si el signo de ambos es positivo, ya que ambos tienen el mismo signo
                rr(a, b, result)
            elif pasosA[indice] < 0: # si el signo de ambos es negativo
                rrr(a, b, result)
        exceso_pasosA = abs(pasosA[indice]) - pasos_comunes
        exceso_pasosB = abs(pasosB[indice]) - pasos_comunes
        giraA(a, b, indice, ((pasosA[indice] > 0) - (pasosA[indice] < 0)) * exceso_pasosA, result)    # (a > 0) - (a < 0) da el signo de a
        giraB(a, b, indice, ((pasosB[indice] > 0) - (pasosB[indice] < 0)) * exceso_pasosB, result)   # Python no tiene función sign
    else:   # No existe sinergia
        giraA(a ,b, indice, pasosA[indice], result)   # gira A
        giraB(a, b, indice, pasosB[indice], result)   # gira B

In [52]:
def situarMax_en_B(a, b, result):   # situa el valor máximo de B en la primera posición de B
    indice = b.index(max(b))
    if indice < len(a)/2:
        steps = indice
    else:
        steps = -(len(b)- indice)
    giraB(a, b, indice, steps, result)

def crecientesA(a, b, result):   # situa los tres valores de A en forma creciente estricta: Ejemplo: 17, 45, 82
    if a[0] < a[1] > a[2] and a[0] < a[2]:        # caso: 1 3 2
        ra(a, b, result)
        sa(a, b, result)
        rra(a, b, result)
    elif a[0] > a[1] < a[2] and a[0] < a[2]:        # caso: 2 1 3
        sa(a, b, result)
    elif a[0] < a[1] > a[2] and a[0] > a[2]:        # caso: 2 3 1
        rra(a, b, result)
    elif a[0] > a[1] < a[2] and a[0] > a[2]:        # caso: 3 1 2
        ra(a, b, result)
    elif a[0] > a[1] > a[2]:                        # caso: 3 2 1
        sa(a, b, result)
        rra(a, b, result)
    return                                          # caso: 1 2 3 en este caso no se hace nada pq ya están ordenados

In [53]:
# Después de ordenar los tres elementos de A en orden creciente estricto con la función crecientesA

def cremallera(a, b, result):   # función que va pasando los números de B a A en forma de cremallera, intercalándolos con los que ya hay en A
    aux = [a[-3], a[-2], a[-1]] # creamos un array aux con los tres últimos valores de A
    while len(aux) > 0:         # Mientras aux tenga elementos
        maximo = max(b + aux)   # calcula el maximo entre b y aux
        if maximo in b:         # si el maximo está en B hacer pa
            pa(a, b, result)
        if maximo in aux:       # si el máximo está en aux hacer rra y aux.pop
            rra(a, b, result)
            aux.pop()
    while len(b) > 0:           # ahora aux está vacío y solo queda hacer pa todo el rato
        pa(a, b, result)

In [54]:
if __name__ == "__main__":
    from random import seed, shuffle
    seed()
    n = 500                             # número de elementos de la pila
    a = list(range(1, n+1)); shuffle(a) # generación aleatoria de la pila A
    a = [403, 185, 148, 467, 213, 260, 497, 198, 215, 478, 332, 188, 477, 122, 307, 284, 243, 133, 210, 63, 236, 1, 286, 80, 79, 290, 4, 373, 29, 326, 154, 149, 134, 47, 410, 192, 486, 333, 341, 64, 494, 411, 294, 350, 381, 451, 105, 226, 305, 83, 176, 293, 264, 455, 404, 461, 292, 287, 201, 248, 318, 76, 48, 285, 349, 480, 496, 70, 255, 73, 491, 412, 357, 13, 336, 152, 394, 363, 484, 311, 306, 88, 46, 11, 327, 263, 177, 240, 151, 34, 367, 162, 202, 14, 130, 457, 89, 140, 303, 474, 17, 387, 170, 230, 238, 124, 492, 23, 15, 112, 55, 456, 41, 430, 182, 200, 12, 157, 272, 490, 57, 108, 228, 100, 121, 24, 324, 131, 391, 344, 67, 110, 42, 5, 301, 313, 446, 295, 126, 2, 266, 483, 417, 85, 335, 399, 30, 288, 214, 168, 99, 96, 414, 253, 278, 141, 366, 321, 346, 343, 86, 65, 153, 328, 347, 234, 137, 72, 50, 211, 308, 468, 138, 187, 207, 406, 374, 244, 377, 107, 413, 454, 227, 442, 111, 10, 135, 407, 420, 25, 195, 180, 289, 18, 174, 171, 393, 265, 245, 159, 340, 81, 235, 448, 173, 239, 21, 376, 416, 460, 423, 184, 178, 354, 206, 476, 358, 267, 431, 466, 383, 94, 304, 225, 425, 452, 370, 405, 445, 380, 252, 433, 269, 339, 386, 66, 459, 183, 487, 385, 49, 246, 20, 172, 369, 249, 75, 164, 322, 93, 158, 160, 302, 256, 118, 444, 331, 91, 128, 143, 348, 60, 16, 38, 258, 359, 166, 362, 262, 314, 495, 78, 283, 196, 428, 251, 102, 427, 229, 45, 378, 139, 58, 106, 129, 92, 127, 175, 9, 488, 232, 61, 470, 219, 310, 39, 3, 297, 395, 271, 439, 365, 422, 485, 443, 223, 95, 274, 101, 31, 400, 179, 415, 209, 27, 84, 82, 401, 315, 479, 489, 398, 247, 500, 197, 355, 69, 205, 163, 220, 298, 250, 281, 345, 216, 408, 421, 212, 43, 447, 259, 193, 337, 435, 317, 382, 53, 117, 33, 296, 40, 465, 279, 37, 54, 254, 98, 145, 475, 309, 473, 44, 218, 299, 19, 120, 51, 424, 462, 116, 449, 300, 74, 8, 97, 62, 147, 275, 368, 330, 257, 123, 481, 437, 150, 68, 104, 392, 132, 432, 190, 419, 181, 379, 231, 155, 119, 7, 499, 372, 291, 342, 353, 472, 463, 390, 352, 351, 273, 87, 450, 436, 35, 493, 222, 375, 270, 6, 56, 438, 109, 217, 199, 338, 189, 409, 36, 434, 316, 169, 312, 113, 441, 22, 389, 268, 261, 453, 418, 319, 77, 458, 224, 426, 323, 71, 115, 191, 356, 125, 440, 334, 282, 237, 320, 397, 203, 32, 396, 329, 280, 156, 186, 371, 167, 136, 194, 165, 103, 114, 384, 26, 498, 233, 471, 221, 325, 276, 429, 59, 364, 90, 146, 142, 402, 52, 361, 208, 469, 204, 388, 161, 277, 464, 144, 482, 242, 360, 241, 28]
    b = []
    a_original = a[:]
    result = []                         # recoje los pasos dados. Ejemplo: result = ['pb','ra','sa','rra']
    pb(a, b, result); pb(a, b, result)  # pasa de A a B la pareja de elementos de A elegidos a[0], a[1]
    while len(a) > 3:
        index_minimosPasos = calculaIndexPasosMinimos()
        giraPilas(a, b, index_minimosPasos, result)
        pb(a, b, result)         # lo pasa de A a B haciendo pb
    situarMax_en_B(a, b, result) # situa el valor máximo de B en la primera posición de B
    crecientesA(a, b, result)    # situa los tres valores de A en forma creciente estricta: Ejemplo: 17, 45, 82
    cremallera(a, b, result)     # función que va pasando los números de B a A en forma de cremallera, intercalándolos con los que ya hay en A
    print(len(result))
    #print(result)

5157


In [55]:
total = [100, 98, 98, 103, 95, 116, 108, 78, 126, 27, 105, 105, 27]
all_min = [i for i, x in enumerate(total) if x == min(total)]   # da un array con todos los index del valor mínimo
all_min

[9, 12]

In [56]:
lista = [10]
lista[-1]

10